In [ ]:
import numpy as np
import os
import time

from lmfit.models import GaussianModel, Model
from sklearn.linear_model import LinearRegression
from sklearn import mixture, metrics

from shapely.geometry import Polygon

from scipy.stats.distributions import norm
from sklearn.neighbors import KernelDensity
from scipy.stats import gaussian_kde

from matplotlib import pyplot as plt
from matplotlib import cm

from skimage.measure import EllipseModel
from matplotlib.patches import Ellipse

from scipy import ndimage as ndi
from scipy.spatial import distance
from numpy import linalg

import skimage
from skimage import io, transform
from skimage.draw import ellipse
from skimage.color import rgb2gray
from skimage import filters
from skimage.morphology import disk
from skimage import filters, util
from skimage.morphology import disk, skeletonize, ball
from skimage import color

from sklearn import preprocessing

from PIL import Image, ImageDraw, ImageFilter, ImageOps
import copy
import cv2
import os
import sys

import matplotlib.pyplot as plt
import matplotlib.gridspec as gridspec
from scipy.ndimage import distance_transform_edt as edt
from skimage.draw import ellipse
from skimage.measure import find_contours, approximate_polygon, subdivide_polygon
import logging
import glob
# from radio_beam.commonbeam import getMinVolEllipse

from src.utils import grainPreprocess, grainShow, grainMark, grainDraw, grainApprox, grainStats, grainMorphology,grainGenerate
from src.cfg import CfgAnglesNames, CfgBeamsNames, CfgDataset


In [ ]:
img=io.imread('data/docs_images/Ultra_Co15-020.jpeg')
preprocess_img=grainPreprocess.image_preprocess(img)

In [ ]:
plt.figure(figsize=(15,15))
plt.imshow(preprocess_img,cmap='gray')

In [ ]:
preprocess_img[:, :, 0].shape

In [ ]:
# io.imsave('cnt.png',cnt_img)
# io.imsave('img_cnt.png',cnt_img)
# io.imsave('contours_binary.png',cnt_img)
io.imsave('binary_inv.png',binary_inv)

In [ ]:
# cnt_img=draw_edges(color.gray2rgb(preprocess_img[:, :, 0]),cnts=cnt)
# cnt_img=draw_edges(color.gray2rgb(img),cnts=cnt)

unsigned_image = util.img_as_ubyte(img)
if len(unsigned_image.shape) < 3:
    unsigned_image = unsigned_image[..., np.newaxis]
denoised = filters.rank.median(unsigned_image, ball(3))
binary = grainPreprocess.do_otsu(denoised)
binary = filters.rank.median(binary, ball(3))
grad = abs(filters.rank.gradient(binary, ball(1)))
bin_grad = (1 - binary + grad) * 127


In [ ]:
binary_inv = filters.rank.median(binary_inv, ball(3))

In [ ]:
cnt_raw=grainMark.get_row_contours(image=preprocess_img)
cnt=grainMark.get_contours(image=preprocess_img)

In [ ]:
def draw_edges(image, cnts, color=(0, 139, 139), r=4, e_width=5, l_width=4):
    """
    :param image: ndarray (width, height, channels)
    :param cnts: ndarray (n_cnts,n,2)
    :param color: tuple (3,)
    :return: ndarray (width, height, channels)
    """
    #
    # рисует на изображении линии по точкам контура cnts
    # линии в стиле x^1->x^2,x^2->x^3 и тд
    #
    new_image = copy.copy(image)

    im = Image.fromarray(new_image)
    draw = ImageDraw.Draw(im)

    for j, cnt in enumerate(cnts):
        if len(cnt) > 1:
            point = cnt[0]
            x1, y1 = point[1], point[0]

            for i, point2 in enumerate(cnt):
                p2 = point2

                x2, y2 = p2[1], p2[0]

                # draw.ellipse((y2 - r, x2 - r, y2 + r, x2 + r), fill=color, width=e_width)
                draw.line((y1, x1, y2, x2), fill=(255, 140, 0), width=l_width)
                x1, y1 = x2, y2

        else:
            continue

    img = np.array(im)

    return img

In [ ]:

cnt_img=draw_edges(color.gray2rgb(preprocess_img[:,:,0]),cnts=cnt_raw,e_width=1, l_width=1)
# cnt_img=draw_edges(preprocess_img,cnts=cnt_raw)


plt.figure(figsize=(15,15))
# plt.imshow(cnt_img,cmap='gray')
plt.imshow(cnt_img)
plt.show()

In [ ]:
plt.figure(figsize=(15,15))
plt.imshow(bin_grad,cmap='gray')

In [ ]:
img1 = io.imread('binary_inv.png')[800:1100,800:1100]
img2 = io.imread('contours_binary.png')[800:1100,800:1100]
plt.figure(figsize=(15, 15))
plt.imshow(img1, cmap='gray')

In [ ]:
io.imsave('binary_inv_cropped.png',img1)
io.imsave('contours_binary_cropped.png',img2)

In [ ]:
io.imsave('preprocessed_cropped_cnt.png',img[800:1100,800:1100])